# Data Input

In [32]:
from pandas import Series,DataFrame
import numpy as np
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')  # link google drive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [33]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [34]:
headerlist = list(range(1,35))
headerlist.append("label")
df = pd.read_csv("/content/gdrive/My Drive/機器學習概論/hw3/ionosphere.data", names=headerlist, sep= ',')
DataFrame(df)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,label
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,0.89391,0.13130,0.81197,0.06723,0.79307,-0.08929,1.00000,-0.02101,0.96639,0.06618,0.87605,0.01155,0.77521,0.06618,0.95378,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,0.96510,0.03281,0.94171,0.07330,0.94625,-0.01326,0.97173,0.00140,0.94834,0.06038,0.92670,0.08412,0.93124,0.10087,0.94520,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,0.94124,0.01766,0.92595,0.04688,0.93954,-0.01461,0.94837,0.02004,0.93784,0.01393,0.91406,0.07677,0.89470,0.06148,0.93988,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,0.89724,-0.03315,0.89061,-0.01436,0.90608,-0.04530,0.91381,-0.00884,0.80773,-0.12928,0.88729,0.01215,0.92155,-0.02320,0.91050,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


# Data Preproocessing

In [35]:
from sklearn.utils import shuffle #shuffle the data
df_shuff = shuffle(df)

# Train-Test-Split(7:3)

In [36]:
from sklearn.model_selection import train_test_split
X = df_shuff.drop(["label"], axis=1)
y = df_shuff["label"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
train = X_train.copy(deep=False)
train["label"] = y_train

# Validation(5-fold)

In [ ]:
def validation(df_origin,k):
  df_tmp = df_origin.copy(deep=False)
  n = int(df_tmp.shape[0]/k)
  # name variable part+i
  for i in range(1,k+1):
    if(i<k):
      globals()['part'+str(i)] = df_tmp.iloc[n*(i-1):n*i]
    else:
      globals()['part'+str(i)] = df_tmp.iloc[n*(i-1):df_tmp.shape[0]]
  
  for i in range(1,k+1):
    globals()['fold'+str(i)] = pd.DataFrame()
    
  for i in range(1,k+1):
    for j in range(1,k+1):
      if(j!=(k+1-i)):
        globals()['fold'+str(i)] = globals()['fold'+str(i)].append(globals()['part'+str(j)])

  for i in range(1,k+1):
    globals()['fold'+str(i)+'_X_train'] = globals()['fold'+str(i)].drop(['label'], axis=1)
    globals()['fold'+str(i)+'_y_train'] = globals()['fold'+str(i)]['label']
    globals()['fold'+str(i)+'_X_test'] = globals()['part'+str(k+1-i)].drop(['label'], axis=1)
    globals()['fold'+str(i)+'_y_test'] = globals()['part'+str(k+1-i)]['label']

In [38]:
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from sklearn.metrics import accuracy_score

def result_func(model, k):
  for i in range(1,k+1):
    model.fit(globals()['fold'+str(i)+'_X_train'], globals()['fold'+str(i)+'_y_train'])
    globals()['fold'+str(i)+'_score'] = model.score(globals()['fold'+str(i)+'_X_test'], globals()['fold'+str(i)+'_y_test'])

  global c_ave, acc_ave, rec_ave, pre_ave, score_ave
  c_ave = 0
  acc_ave = 0
  rec_ave = 0
  pre_ave = 0
  score_ave = 0
  
  for i in range(1,k+1):
    score_ave += globals()['fold'+str(i)+'_score']

  score_ave = round(score_ave / k,4)

# Grid Search(Poly)

In [39]:
from sklearn.svm import SVC
from tabulate import tabulate

# use grid search to find the best hyperparameter pair for poly and rbf
validation(train,5)
print("grid search for SVM_poly")
best_score_poly = 0
step_poly = 0
for degree in [0.1, 1, 10]:
  for gamma in [0.1, 1, 10]:
    for coef in [0.1, 1, 10]:
      step_poly+=1
      svm_poly_tmp = SVC(kernel='poly', degree=degree, gamma=gamma, coef0=coef)
      result_func(svm_poly_tmp,5)
      globals()['parameter'+str(step_poly)] = [degree, gamma, coef, score_ave]
      if score_ave > best_score_poly:
        best_score_poly = score_ave
        best_parameter_poly = [degree, gamma, coef]

print("best score: ", best_score_poly)
print("best parameter: ", best_parameter_poly)


grid search for SVM_poly
best score:  0.8653
best parameter:  [1, 1, 0.1]


# Question(Poly)

In [40]:
scores = []
for i in range(1, step_poly+1):
  tmp1 = globals()['parameter'+str(i)].copy()
  scores.append(tmp1)

scores.sort(key = lambda s: s[3], reverse = True)
table_head = ["degree", "gamma", "coef", "accuracy", "rank"]
table_data = []

for i in range(1, step_poly+1):
  tmp2 = scores[i-1].copy()
  tmp2.append(i)
  table_data.append(tmp2)

print("compare different parameter for SVM_Poly:")
print("\n",tabulate(table_data, headers=table_head, tablefmt="grid"))

compare different parameter for SVM_Poly:

 +----------+---------+--------+------------+--------+
|   degree |   gamma |   coef |   accuracy |   rank |
+==========+=========+========+============+========+
|      1   |     1   |    0.1 |     0.8653 |      1 |
+----------+---------+--------+------------+--------+
|      1   |     1   |    1   |     0.8653 |      2 |
+----------+---------+--------+------------+--------+
|      1   |     1   |   10   |     0.8653 |      3 |
+----------+---------+--------+------------+--------+
|     10   |    10   |    0.1 |     0.8612 |      4 |
+----------+---------+--------+------------+--------+
|      1   |     0.1 |    0.1 |     0.8571 |      5 |
+----------+---------+--------+------------+--------+
|      1   |     0.1 |    1   |     0.8571 |      6 |
+----------+---------+--------+------------+--------+
|      1   |     0.1 |   10   |     0.8571 |      7 |
+----------+---------+--------+------------+--------+
|      1   |    10   |    0.1 |     0.

# Grid Search(RBF)

In [41]:
print("grid search for SVM_rbf")
best_score_rbf = 0
step_rbf = 0
for gamma in [0.1, 1, 10]:
  step_rbf+=1
  svm_rbf_tmp = SVC(kernel='rbf', gamma=gamma)
  result_func(svm_rbf_tmp,5)
  globals()['parameter'+str(step_rbf)] = [gamma, score_ave]
  if score_ave > best_score_rbf:
    best_score_rbf = score_ave
    best_parameter_rbf = [gamma]

print("best score: ", best_score_rbf)
print("best parameter: ", best_parameter_rbf)

grid search for SVM_rbf
best score:  0.9224
best parameter:  [0.1]


# Question(RBF)

In [42]:
scores = []
for i in range(1, step_rbf+1):
  tmp1 = globals()['parameter'+str(i)].copy()
  scores.append(tmp1)
scores.sort(key = lambda s: s[1], reverse = True)
table_head = ["gamma", "accuracy", "rank"]
table_data = []
for i in range(1, step_rbf+1):
  tmp2 = scores[i-1].copy()
  tmp2.append(i)
  table_data.append(tmp2)

print("compare different parameter for SVM_RBF:")
print("\n",tabulate(table_data, headers=table_head, tablefmt="grid"))

compare different parameter for SVM_RBF:

 +---------+------------+--------+
|   gamma |   accuracy |   rank |
+=========+============+========+
|     0.1 |     0.9224 |      1 |
+---------+------------+--------+
|     1   |     0.8939 |      2 |
+---------+------------+--------+
|    10   |     0.6204 |      3 |
+---------+------------+--------+


# SVM Construct

In [ ]:
svm_linear = SVC(kernel='linear')
svm_poly = SVC(kernel='poly', degree=best_parameter_poly[0], gamma=best_parameter_poly[1], coef0=best_parameter_poly[2])
svm_rbf = SVC(kernel='rbf', gamma=best_parameter_rbf[0])

# Result

In [ ]:
# svm_linear result
svm_linear.fit(X_train, y_train)
svm_linear_pred = svm_linear.predict(X_test)
c_linear = confusion_matrix(y_test, svm_linear_pred)
acc_linear = round(accuracy_score(y_test, svm_linear_pred)*100,2)
rec_linear = sklearn.metrics.recall_score(y_test, svm_linear_pred, average=None)
pre_linear = sklearn.metrics.precision_score(y_test, svm_linear_pred, average=None)
print("svm with linear:")
print("\nAccuracy: {}%".format(acc_linear))
print("confusion matrix:\n", c_linear)
table_head = ["class Name", "Sensitivity(Recall)", "Precision"]
table_data = [["b", "{}%".format(round(rec_linear[0]*100,2)), "{}%".format(round(pre_linear[0]*100,2))],
        ["g", "{}%".format(round(rec_linear[1]*100,2)), "{}%".format(round(pre_linear[1]*100,2))]]
print("\n",tabulate(table_data, headers=table_head, tablefmt="grid"))

# svm_poly result
svm_poly.fit(X_train, y_train)
svm_poly_pred = svm_poly.predict(X_test)
c_poly = confusion_matrix(y_test, svm_poly_pred)
acc_poly = round(accuracy_score(y_test, svm_poly_pred)*100,2)
rec_poly = sklearn.metrics.recall_score(y_test, svm_poly_pred, average=None)
pre_poly = sklearn.metrics.precision_score(y_test, svm_poly_pred, average=None)
print("\nsvm with poly:")
print("\nAccuracy: {}%".format(acc_poly))
print("confusion matrix:\n", c_poly)
table_head = ["class Name", "Sensitivity(Recall)", "Precision"]
table_data = [["b", "{}%".format(round(rec_poly[0]*100,2)), "{}%".format(round(pre_poly[0]*100,2))],
        ["g", "{}%".format(round(rec_poly[1]*100,2)), "{}%".format(round(pre_poly[1]*100,2))]]
print("\n",tabulate(table_data, headers=table_head, tablefmt="grid"))

# svm_rbf result
svm_rbf.fit(X_train, y_train)
svm_rbf_pred = svm_rbf.predict(X_test)
c = confusion_matrix(y_test, svm_rbf_pred)
acc = round(accuracy_score(y_test, svm_rbf_pred)*100,2)
rec = sklearn.metrics.recall_score(y_test, svm_rbf_pred, average=None)
pre = sklearn.metrics.precision_score(y_test, svm_rbf_pred, average=None)
print("\nsvm with rbf:")
print("\nAccuracy: {}%".format(acc))
print("confusion matrix:\n", c)
table_head = ["class Name", "Sensitivity(Recall)", "Precision"]
table_data = [["b", "{}%".format(round(rec[0]*100,2)), "{}%".format(round(pre[0]*100,2))],
        ["g", "{}%".format(round(rec[1]*100,2)), "{}%".format(round(pre[1]*100,2))]]
print("\n",tabulate(table_data, headers=table_head, tablefmt="grid"))

svm with linear:

Accuracy: 89.62%
confusion matrix:
 [[32  7]
 [ 4 63]]

 +--------------+-----------------------+-------------+
| class Name   | Sensitivity(Recall)   | Precision   |
+==============+=======================+=============+
| b            | 82.05%                | 88.89%      |
+--------------+-----------------------+-------------+
| g            | 94.03%                | 90.0%       |
+--------------+-----------------------+-------------+

svm with poly:

Accuracy: 86.79%
confusion matrix:
 [[27 12]
 [ 2 65]]

 +--------------+-----------------------+-------------+
| class Name   | Sensitivity(Recall)   | Precision   |
+==============+=======================+=============+
| b            | 69.23%                | 93.1%       |
+--------------+-----------------------+-------------+
| g            | 97.01%                | 84.42%      |
+--------------+-----------------------+-------------+

svm with rbf:

Accuracy: 94.34%
confusion matrix:
 [[34  5]
 [ 1 66]]

 +-------